# Imports

In [1]:
using Pkg

In [2]:
Pkg.add("MacroTools");
Pkg.add("ChainRulesCore");
Pkg.add("Zygote");
Pkg.add("Revise")
Pkg.add("IRTools")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
Precompiling project...
  ✓ Zygote
  ✓ Flux
  2 dependencies successfully precompiled in 15 seconds. 84 already precompiled.
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
   Installed Zygote ─ v0.6.49
    Updating `~/.julia/environments/v1.8/Project.toml`
  [e88e6eb3] ~ Zygote v0.6.47 `~/.julia/dev/Zygote` ⇒ v0.6.49
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [e88e6eb3] ~ Zygote v0.6.47 `~/.julia/dev/Zygote` ⇒ v0.6.49
Precompiling project...
  ✓ Zygote
  ✓ Flux
  2 dependencies successfully precompiled in 15 seconds. 84 already precompiled.
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
 

In [15]:
using MacroTools: prewalk, postwalk
using ChainRulesCore
#using Zygote
using IRTools
using IRTools: func, var
using Revise

In [4]:
Pkg.develop("Zygote")

   Resolving package versions...
    Updating `~/.julia/environments/v1.8/Project.toml`
  [e88e6eb3] ~ Zygote v0.6.49 ⇒ v0.6.47 `~/.julia/dev/Zygote`
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [e88e6eb3] ~ Zygote v0.6.49 ⇒ v0.6.47 `~/.julia/dev/Zygote`


In [2]:
using Zygote

In [5]:
#Zygote.gradient(a -> sin(a), 3)

In [6]:
#using Zygote

In [3]:
function pow(b, p)
    if p == 0
        return 1
    end
    if p % 2 == 0
        return pow(b ^ 2, p // 2)
    end
    p = p - 1
    return b * pow(b, p) 
end

pow (generic function with 1 method)

In [4]:
pow_ir = Zygote.@code_ir pow(3, 5)

1: (%1, %2, %3)
  %4 = %3 == 0
  br 2 unless %4
  return 1
2:
  %5 = %3 % 2
  %6 = %5 == 0
  br 4 unless %6
  br 3
3:
  %7 = Core.apply_type(Base.Val, 2)
  %8 = (%7)()
  %9 = Base.literal_pow(Main.:^, %2, %8)
  %10 = %3 // 2
  %11 = Main.pow(%9, %10)
  return %11
4:
  %12 = %3 - 1
  %13 = Main.pow(%2, %12)
  %14 = %2 * %13
  return %14

In [6]:
pow_ir[var(5)]

IRTools.Inner.Statement(:(%3 % 2), Any, 3)

In [7]:
function crc_test(x)
    a = sin(x)
    b = 3 + a
    c = b^2 + a
    return c
end

crc_test (generic function with 1 method)

easy_test (generic function with 1 method)

In [9]:
crc_ir = Zygote.@code_ir crc_test(3)

1: (%1, %2)
  %3 = Main.sin(%2)
  %4 = 3 + %3
  %5 = Core.apply_type(Base.Val, 2)
  %6 = (%5)()
  %7 = Base.literal_pow(Main.:^, %4, %6)
  %8 = %7 + %3
  return %8

In [10]:
typeof(crc_ir[var(5)].expr)

Expr

In [11]:
crc_ir

1: (%1, %2)
  %3 = Main.sin(%2)
  %4 = 3 + %3
  %5 = Core.apply_type(Base.Val, 2)
  %6 = (%5)()
  %7 = Base.literal_pow(Main.:^, %4, %6)
  %8 = %7 + %3
  return %8

In [12]:
#IRTools.insertafter!(crc_ir, var(8), :(println("Last")))

In [13]:
#typeof(crc_ir[var(3)].expr.args[2])

In [16]:
out = IRTools.insertafter!(crc_ir, var(8), IRTools.Statement(:($(var(4)) * $(var(8))), Any, 0))

%9

In [17]:
IRTools.insertafter!(crc_ir, out, :(println("intermediate", $(out))))

%10

In [18]:
#insert!(crc_ir, var(4), :(println("Hello World!")))

In [19]:
crc_ir

1: (%1, %2)
  %3 = Main.sin(%2)
  %4 = 3 + %3
  %5 = Core.apply_type(Base.Val, 2)
  %6 = (%5)()
  %7 = Base.literal_pow(Main.:^, %4, %6)
  %8 = %7 + %3
  %9 = %4 * %8
  %10 = println("intermediate", %9)
  return %8

In [20]:
crc_f = func(crc_ir)

##312 (generic function with 1 method)

In [21]:
crc_f(nothing, 3)

intermediate31.435559193278646


10.007754913093887

In [22]:
# z = x * y + sin(x)

# a = sin(x)
# b = x * y
# z = a + b
# return z

# gz = 1
# ga = 1
# gb = 1
# gx = y
# gy = x

# Starting from the end:
# return z => gz = 1
# z = a + b => ga += gz, gb += gz
# b = x * y => gx += y * gb; gy += x * ga
# a = sin(x) => gx += cos(x) * ga

# so gx = y + cos(x); gy = x

differentiate (generic function with 1 method)

In [257]:
c = differentiate(easy_test, 3, 5)

6 IRTools.Inner.Statement(:(%4 + %5), Any, 1) 4
add
5 IRTools.Inner.Statement(:(Main.sin(%2)), Any, 1) 8
4 IRTools.Inner.Statement(:(%2 * %3), Any, 1) 12
Dict{Any, Any}(%6 => %7, %3 => %21, %4 => %10, %5 => %11, %2 => %19)
1: (%1, %2, %3)
  %4 = %2 * %3
  %5 = Main.sin(%2)
  %6 = %4 + %5
  %7 = 1
  %8 = 0
  %9 = 0
  %10 = %8 + %7
  %11 = %9 + %7
  %12 = 0
  %13 = cos(%2)
  %14 = %11 * %13
  %15 = %12 + %14
  %16 = 0
  %17 = 0
  %18 = %10 * %3
  %19 = %15 + %18
  %20 = %10 * %2
  %21 = %17 + %20
  %22 = Core.tuple(%6, %19, %21)
  return %22

##402 (generic function with 1 method)

In [258]:
c(nothing, 3, 5)

(15.141120008059866, 4.010007503399555, 3)

In [259]:
easy_test(3, 5)

15.141120008059866

In [269]:
function basic_condition(a, b)
    if a > b
        return a + b
    else
        return a * b
    end
end

basic_condition (generic function with 1 method)

In [272]:
basic_condition(5, 3)

8

In [275]:
cond_ir = (IRTools.@code_ir basic_condition(5, 3))

1: (%1, %2, %3)
  %4 = %2 > %3
  br 3 unless %4
2:
  %5 = %2 + %3
  return %5
3:
  %6 = %2 * %3
  return %6

In [278]:
cond_ir

1: (%1, %2, %3)
  %4 = %2 > %3
  br 3 unless %4
2:
  %5 = %2 + %3
  return %5
3:
  %6 = %2 * %3
  return %6

In [184]:
stripped_ir = IRTools.IR()

1:

In [150]:
ov = IRTools.push!(stripped_ir, :(1, 2, 3))
IRTools.return!(stripped_ir, ov)

1:
  %1 = (1, 2, 3)
  return %1

In [152]:
func(stripped_ir)()

Internal error: encountered unexpected error during compilation of ##368:
ErrorException("unsupported or misplaced expression "tuple" in function ##368")
ijl_errorf at /cache/build/default-amdci5-0/julialang/julia-release-1-dot-8/src/rtutils.c:77
emit_expr at /cache/build/default-amdci5-0/julialang/julia-release-1-dot-8/src/codegen.cpp:5044
emit_ssaval_assign at /cache/build/default-amdci5-0/julialang/julia-release-1-dot-8/src/codegen.cpp:4315
emit_stmtpos at /cache/build/default-amdci5-0/julialang/julia-release-1-dot-8/src/codegen.cpp:4562 [inlined]
emit_function at /cache/build/default-amdci5-0/julialang/julia-release-1-dot-8/src/codegen.cpp:7416
jl_emit_code at /cache/build/default-amdci5-0/julialang/julia-release-1-dot-8/src/codegen.cpp:7778
jl_emit_codeinst at /cache/build/default-amdci5-0/julialang/julia-release-1-dot-8/src/codegen.cpp:7823
_jl_compile_codeinst at /cache/build/default-amdci5-0/julialang/julia-release-1-dot-8/src/jitlayers.cpp:119
jl_generate_fptr_impl at /cache/b

LoadError: unsupported or misplaced expression tuple

In [157]:
function list(a, b)
    out = (a, b)
    return out
end

list (generic function with 1 method)

In [202]:
function list(a, b)
    return (a, b)
end

list (generic function with 1 method)

In [226]:
list_ir = (IRTools.@code_ir list(3, 5))

1: (%1, %2, %3)
  %4 = Core.tuple(%2, %3)
  return %4

In [246]:
list_ir[var(4)] = IRTools.Inner.Statement(:($(GlobalRef(Core, :tuple))($(var(1)), $(var(3)))))

IRTools.Inner.Statement(:(Core.tuple(%1, %3)), Any, 0)

In [247]:
list_ir

1: (%1, %2, %3)
  %4 = Core.tuple(%1, %3)
  return %4

In [245]:
GlobalRef(Core, :tuple)

:(Core.tuple)

In [248]:
original = dump(list_ir)

IRTools.Inner.IR
  defs: Array{Tuple{Int64, Int64}}((4,))
    1: Tuple{Int64, Int64}
      1: Int64 1
      2: Int64 -1
    2: Tuple{Int64, Int64}
      1: Int64 1
      2: Int64 -2
    3: Tuple{Int64, Int64}
      1: Int64 1
      2: Int64 -3
    4: Tuple{Int64, Int64}
      1: Int64 1
      2: Int64 1
  blocks: Array{IRTools.Inner.BasicBlock}((1,))
    1: IRTools.Inner.BasicBlock
      stmts: Array{IRTools.Inner.Statement}((1,))
        1: IRTools.Inner.Statement
          expr: Expr
            head: Symbol call
            args: Array{Any}((3,))
              1: GlobalRef
                mod: Module Core
                name: Symbol tuple
              2: IRTools.Inner.Variable
                id: Int64 1
              3: IRTools.Inner.Variable
                id: Int64 3
          type: Any
          line: Int64 0
      args: Array{Any}((3,))
        1: IRTools.Inner.Variable
          id: Int64 1
        2: IRTools.Inner.Variable
          id: Int64 2
        3: IRTools.Inner.Var

          def: Method
            name: Symbol ##384
            module: Module IRTools.Inner
            file: Symbol /home/robert/.julia/packages/IRTools/017wp/src/eval.jl
            line: Int32 18
            primary_world: UInt64 0x0000000000007f44
            deleted_world: UInt64 0xffffffffffffffff
            sig: Tuple{IRTools.Inner.var"###384", Any, Any, Any} <: Any
            specializations: SimpleVector
              1: Core.MethodInstance#= circular reference @-2 =#
              2: Nothing nothing
              3: Nothing nothing
              4: Nothing nothing
              5: Nothing nothing
              6: Nothing nothing
              7: Nothing nothing
              8: Nothing nothing
            speckeyset: Array{UInt8}((32,)) UInt8[0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00  …  0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00]
            slot_syms: String "#self#\0arg1\0arg2\0arg3\0"
            external_mt: #undef
            source:

          file: Symbol In[202]
          line: Int64 2
          inlined_at: Int64 0
      slotnames: Array{Symbol}((3,))
        1: Symbol #self#
        2: Symbol a
        3: Symbol b
      slotflags: Array{UInt8}((3,)) UInt8[0x00, 0x08, 0x08]
      slottypes: Nothing nothing
      rettype: Any
      parent: Nothing nothing
      edges: Nothing nothing
      min_world: UInt64 0x0000000000000001
      max_world: UInt64 0xffffffffffffffff
      inferred: Bool false
      inlineable: Bool false
      propagate_inbounds: Bool false
      pure: Bool false
      constprop: UInt8 0x00
      purity: UInt8 0x00
    nargs: Int64 3
    sparams: empty SimpleVector


In [249]:
func(list_ir)(nothing, 3, 5)

(nothing, 5)

In [169]:
IRTools.Inner.Statement(:(Core.tuple($(var(2)), $(var(3)))))

IRTools.Inner.Statement(:(Core.tuple(%2, %3)), Any, 0)

In [201]:
list_ir[var(4)] = IRTools.Inner.Statement(:(Core.tuple($(var(2)), $(var(3)))))
list_id = func(list_ir)
list_id(nothing, 2, 4)

Internal error: encountered unexpected error in runtime:
AssertionError(msg="argextype only works on argument-position values")
argextype at ./compiler/optimize.jl:353
argextype at ./compiler/optimize.jl:335 [inlined]
argextype at ./compiler/optimize.jl:335 [inlined]
call_sig at ./compiler/ssair/inlining.jl:950
process_simple! at ./compiler/ssair/inlining.jl:1120
assemble_inline_todo! at ./compiler/ssair/inlining.jl:1359
ssa_inlining_pass! at ./compiler/ssair/inlining.jl:82
jfptr_ssa_inlining_passNOT._19267.clone_1 at /home/robert/.julia/juliaup/julia-1.8.1+0.x64/lib/julia/sys.so (unknown line)
_jl_invoke at /cache/build/default-amdci5-0/julialang/julia-release-1-dot-8/src/gf.c:2367 [inlined]
ijl_apply_generic at /cache/build/default-amdci5-0/julialang/julia-release-1-dot-8/src/gf.c:2549
run_passes at ./compiler/optimize.jl:539
optimize at ./compiler/optimize.jl:504 [inlined]
_typeinf at ./compiler/typeinfer.jl:257
typeinf at ./compiler/typeinfer.jl:213
typeinf_ext at ./compiler/typein

LoadError: unsupported or misplaced expression .

In [189]:
function notlist(x)
    return x * 3 + 1
end

notlist (generic function with 1 method)

In [192]:
notlist_ir = (IRTools.@code_ir notlist(3))

1: (%1, %2)
  %3 = %2 * 3
  %4 = %3 + 1
  return %4

In [194]:
notlist_ir[var(4)] = IRTools.Inner.Statement(:(Core.tuple($(var(2)), $(var(3)))))

IRTools.Inner.Statement(:(Core.tuple(%2, %3)), Any, 0)

In [196]:
func(notlist_ir)(nothing, 1)

Internal error: encountered unexpected error in runtime:
AssertionError(msg="argextype only works on argument-position values")
argextype at ./compiler/optimize.jl:353
argextype at ./compiler/optimize.jl:335 [inlined]
argextype at ./compiler/optimize.jl:335 [inlined]
call_sig at ./compiler/ssair/inlining.jl:950
process_simple! at ./compiler/ssair/inlining.jl:1120
assemble_inline_todo! at ./compiler/ssair/inlining.jl:1359
ssa_inlining_pass! at ./compiler/ssair/inlining.jl:82
jfptr_ssa_inlining_passNOT._19267.clone_1 at /home/robert/.julia/juliaup/julia-1.8.1+0.x64/lib/julia/sys.so (unknown line)
_jl_invoke at /cache/build/default-amdci5-0/julialang/julia-release-1-dot-8/src/gf.c:2367 [inlined]
ijl_apply_generic at /cache/build/default-amdci5-0/julialang/julia-release-1-dot-8/src/gf.c:2549
run_passes at ./compiler/optimize.jl:539
optimize at ./compiler/optimize.jl:504 [inlined]
_typeinf at ./compiler/typeinfer.jl:257
typeinf at ./compiler/typeinfer.jl:213
typeinf_ext at ./compiler/typein

LoadError: unsupported or misplaced expression .

In [113]:
list_ir.blocks[1].stmts[1]

IRTools.Inner.Statement(:(Base.vect()), Any, 1)

##378 (generic function with 1 method)

Internal error: encountered unexpected error in runtime:
AssertionError(msg="argextype only works on argument-position values")
argextype at ./compiler/optimize.jl:353
argextype at ./compiler/optimize.jl:335 [inlined]
argextype at ./compiler/optimize.jl:335 [inlined]
call_sig at ./compiler/ssair/inlining.jl:950
process_simple! at ./compiler/ssair/inlining.jl:1120
assemble_inline_todo! at ./compiler/ssair/inlining.jl:1359
ssa_inlining_pass! at ./compiler/ssair/inlining.jl:82
jfptr_ssa_inlining_passNOT._19267.clone_1 at /home/robert/.julia/juliaup/julia-1.8.1+0.x64/lib/julia/sys.so (unknown line)
_jl_invoke at /cache/build/default-amdci5-0/julialang/julia-release-1-dot-8/src/gf.c:2367 [inlined]
ijl_apply_generic at /cache/build/default-amdci5-0/julialang/julia-release-1-dot-8/src/gf.c:2549
run_passes at ./compiler/optimize.jl:539
optimize at ./compiler/optimize.jl:504 [inlined]
_typeinf at ./compiler/typeinfer.jl:257
typeinf at ./compiler/typeinfer.jl:213
typeinf_ext at ./compiler/typein

LoadError: unsupported or misplaced expression .

In [62]:
IRTools.@code_ir easy_test(3, 5)

1: (%1, %2, %3)
  %4 = %2 * %3
  %5 = Main.sin(%2)
  %6 = %4 + %5
  return %6

In [ ]:
@IRToo

In [27]:
Zygote.gradient(easy_test, 3, 5)

(4.010007503399555, 3.0)

In [118]:
using IRTools: returnvalue

In [123]:
crc_ir.blocks[1].branches

1-element Vector{IRTools.Inner.Branch}:
 return %8

In [134]:
IRTools.returnvalue(IRTools.blocks(crc_ir)[1])

%8

In [80]:
typeof(crc_f)

IRTools.Inner.var"###312"

In [33]:
crc_f

##312 (generic function with 1 method)

In [22]:
crc_test(4)

4.275132549748808

In [49]:
l = length(crc_ir)

5

In [67]:
for i=1:(length(crc_ir)+5)
    try
        println(crc_ir[var(i)])
    catch e
        println(e)
    end
end

ErrorException("No such variable %1")
ErrorException("No such variable %2")
IRTools.Inner.Statement(:(Main.sin(%2)), Any, 1)
IRTools.Inner.Statement(:(3 + %3), Any, 2)
IRTools.Inner.Statement(:(Core.apply_type(Base.Val, 2)), Any, 3)
IRTools.Inner.Statement(:((%5)()), Any, 3)
IRTools.Inner.Statement(:(Base.literal_pow(Main.:^, %4, %6)), Any, 3)
ErrorException("No such variable %8")
ErrorException("No such variable %9")
ErrorException("No such variable %10")


In [72]:
crc_ir.blocks[1].stmts

5-element Vector{IRTools.Inner.Statement}:
 IRTools.Inner.Statement(:(Main.sin(%2)), Any, 1)
 IRTools.Inner.Statement(:(3 + %3), Any, 2)
 IRTools.Inner.Statement(:(Core.apply_type(Base.Val, 2)), Any, 3)
 IRTools.Inner.Statement(:((%5)()), Any, 3)
 IRTools.Inner.Statement(:(Base.literal_pow(Main.:^, %4, %6)), Any, 3)

In [50]:
crc_ir[var(l)]

IRTools.Inner.Statement(:(Core.apply_type(Base.Val, 2)), Any, 3)

In [18]:
typeof(@code_ir crc_test(4))

IRTools.Inner.IR

In [ ]:
for e in @core_ir  crc_test(4)

In [22]:
IRTools.postwalk(t -> begin println(t); return t end, @code_ir crc_test(4))

Main.sin
%2
Main.sin(%2)
Main.:+
3
%3
3 + %3
Core.apply_type
Base.Val
2
Core.apply_type(Base.Val, 2)
%5
(%5)()
Base.literal_pow
Main.:^
%4
%6
Base.literal_pow(Main.:^, %4, %6)
nothing
%7
1: (%1, %2)
  %3 = Main.sin(%2)
  %4 = 3 + %3
  %5 = Core.apply_type(Base.Val, 2)
  %6 = (%5)()
  %7 = Base.literal_pow(Main.:^, %4, %6)
  return %7


1: (%1, %2)
  %3 = Main.sin(%2)
  %4 = 3 + %3
  %5 = Core.apply_type(Base.Val, 2)
  %6 = (%5)()
  %7 = Base.literal_pow(Main.:^, %4, %6)
  return %7

In [43]:
meta(typeof(gcd))

LoadError: BoundsError: attempt to access Core.SimpleVector at index [1]

In [26]:
meta(Tuple{typeof(gcd), Int, Int})

Metadata for gcd(a::T, b::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at intfuncs.jl:50

In [17]:
meta(gcd)

LoadError: type #gcd has no field parameters

In [49]:
Zygote._generate_pullback_via_decomposition(a -> sin(a))

LoadError: type #13#14 has no field parameters

(-0.9899924966004454,)

In [ ]:
edit(Zygote)

Vim: Warning: Output is not to a terminal
Vim: Warning: Input is not from a terminal


</Zygote/PD12J/src.jl" [readonly] 82L, 2240Cmodule Zygote

using LinearAlgebra, Statistics
using LinearAlgebra: copytri!, AbstractTriangular

import ZygoteRules: @adjoint, @adjoint!, AContext, adjoint, _pullback, pullback,literal_getproperty, literal_getfield

using ChainRulesCore
using ChainRules: ChainRules, rrule, unthunk, canonicalize
using IRTools
using MacroTools, Requires
using MacroTools: @forward

import Distributed: pmap, CachingPool, workers
export Params, withgradient, gradient, withjacobian, jacobian, hessian, diaghesssian, pullback, pushforward, @code_adjoint
export rrule_via_ad

const Numeric{T<:Number} = Union{T, AbstractArray{<:T}}

include("deprecated.jl")
include("tools/buffer.jl")
include("tools/builtins.jl")

include("forward/Forward.jl")
using .Forward

include("compiler/reverse.jl")1,1Top

In [1]:
y = x^2 + 3

LoadError: UndefVarError: x not defined

In [2]:
y = :(x^2 + 3)

:(x ^ 2 + 3)

In [3]:
:((x^2) + 3)

:(x ^ 2 + 3)

In [4]:
y.args

3-element Vector{Any}:
  :+
  :(x ^ 2)
 3

In [8]:
y.head

:call

In [24]:
(:(5x)).args

3-element Vector{Any}:
  :*
 5
  :x

In [30]:
l = [1, 2, 3, 4]

4-element Vector{Int64}:
 1
 2
 3
 4

In [25]:
eval(Expr(:call, :+, 2, 3, 4))

9

In [55]:
function d(expr, wrt)
    if (expr == wrt)
        return 1
    elseif (expr isa Symbol || expr isa Number)
        return 0
    elseif (expr.head == :call && first(expr.args) == :+)
        return sum(map(a -> d(a, wrt), expr.args[1:end]))
    end
end

d (generic function with 1 method)

In [58]:
d(:(x + 3), :x)

1

In [78]:
ex = :(f(1, 2, 3))

:(f(1, 2, 3))

In [80]:
@capture(ex, func_(args__))

true

In [69]:
:(struct Foo
    x::Int64
    y
    end)

:(struct Foo
      #= In[69]:2 =#
      x::Int64
      #= In[69]:3 =#
      y
  end)

In [88]:
postwalk(a -> a isa Integer ? a // 2 : a, ex)

:(f(1//2, 1//1, 3//2))

In [89]:
polynomial = :(3x^4 + 5y^2 + 9z^3)

:(3 * x ^ 4 + 5 * y ^ 2 + 9 * z ^ 3)

In [134]:
c = e -> @capture(e, a_Number * v_Symbol ^ p_Number) ? :($p*$a*$v^($p-1)) : e

#49 (generic function with 1 method)

In [135]:
c(:(3x ^ 4))

:(4 * 3 * x ^ (4 - 1))

In [136]:
postwalk(c, polynomial)

:(4 * 3 * x ^ (4 - 1) + 2 * 5 * y ^ (2 - 1) + 3 * 9 * z ^ (3 - 1))

In [ ]:
@capture(ex, )

In [ ]:
macro foo(ex)
  postwalk(ex) do x
    @capture(x, some_pattern) || return x
    return new_x
  end
end


In [154]:
macro declare_then_print(ex)
    return :(
        println("I am about to evaluate $ex"),
        ex
        )
end

@declare_then_print (macro with 1 method)

In [151]:
@macroexpand @declare_then_print(:(3 + 5 * :x))

:(Main.println("I am about to evaluate $(Main.ex)") $ Main.ex)

In [155]:
@declare_then_print(:(3 + 5))

I am about to evaluate f(1, 2, 3)


(nothing, :(f(1, 2, 3)))

In [157]:
## ChainRulesCore

   Resolving package versions...
   Installed ChainRulesCore ─ v1.15.5
    Updating `~/.julia/environments/v1.8/Project.toml`
  [d360d2e6] + ChainRulesCore v1.15.5
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [d360d2e6] ↑ ChainRulesCore v1.15.4 ⇒ v1.15.5
Precompiling project...
  ✓ ChainRulesCore
  ✓ ChangesOfVariables
  ✓ AbstractFFTs
  ✓ Optimisers
  ✓ LogExpFunctions
  ✓ NNlib
  ✓ StatsBase
  ✓ MLUtils
  ✓ SpecialFunctions
  ✓ ChainRules
  ✓ DiffRules
  ✓ ForwardDiff
  ✓ Zygote
  ✓ CUDA
  ✓ NNlibCUDA
  ✓ Flux
  16 dependencies successfully precompiled in 44 seconds. 66 already precompiled.


In [42]:
function crc_test(x)
    a = sin(x)
    b = 3 + a
    c = b^2
    return c
end

crc_test (generic function with 1 method)

In [43]:
Zygote.gradient(x -> crc_test(x), 1)

(4.1511112620345205,)

In [193]:
@macroexpand @scalar_rule(sin(x), cos(x))

quote
    #= /home/robert/.julia/packages/ChainRulesCore/Z4Jry/src/rule_definition_tools.jl:103 =#
    if !(sin isa ChainRulesCore.Type) && ChainRulesCore.fieldcount(ChainRulesCore.typeof(sin)) > 0
        #= /home/robert/.julia/packages/ChainRulesCore/Z4Jry/src/rule_definition_tools.jl:104 =#
        ChainRulesCore.throw(ChainRulesCore.ArgumentError("@scalar_rule cannot be used on closures/functors (such as $(sin))"))
    end
    #= /home/robert/.julia/packages/ChainRulesCore/Z4Jry/src/rule_definition_tools.jl:111 =#
    begin
        function (ChainRulesCore.ChainRulesCore).derivatives_given_output(Ω, ::(ChainRulesCore.Core).Typeof(sin), x::Number)
            #= In[193]:1 =#
            return ((cos(x),),)
        end
    end
    #= /home/robert/.julia/packages/ChainRulesCore/Z4Jry/src/rule_definition_tools.jl:112 =#
    begin
        function (ChainRulesCore.ChainRulesCore).frule((_, var"##Δ1#328"), ::(ChainRulesCore.Core).Typeof(sin), x::Number)
            #= In[193]:1 =#
       

In [15]:
function derivatives_given_output2(Ω, ::(ChainRulesCore.Core).Typeof(sin), x::Number)
    return ((cos(x),),)
end

derivatives_given_output2 (generic function with 1 method)

In [17]:
derivatives_given_output2(sin(π), sin, π)

((-1.0,),)

In [18]:
function frule2((_, Δ1), ::(ChainRulesCore.Core).Typeof(sin), x::Number)
    Ω = sin(x)
    ((∂f1∂x1,),) = derivatives_given_output2(Ω, sin, x::Number)
    return (Ω, (identity)(∂f1∂x1 * Δ1))
end

frule2 (generic function with 1 method)

In [23]:
frule2((0, 100), sin, 3)

(0.1411200080598672, -98.99924966004454)

In [25]:
function rrule2(::(ChainRulesCore.Core).Typeof(sin), x::Number)
    Ω = sin(x)
    proj_x = ChainRulesCore.ProjectTo(x)
    ((∂f1∂x1,),) = derivatives_given_output2(Ω, sin, x::Number)
    return (Ω, begin
                function sin_pullback(Δ1)
                    return (ChainRulesCore.NoTangent(), proj_x(ChainRulesCore.conj(∂f1∂x1) * Δ1))
                end
            end)
end

rrule2 (generic function with 1 method)

In [32]:
rrule2(sin, 3)[2](100)

(NoTangent(), -98.99924966004454)

In [163]:
@scalar_rule(sin(x), cos(x))

In [181]:
@scalar_rule(+(x, y), (1.0, 1.))

In [164]:
x = 3
a, a_pullback = rrule(sin, x)

(0.1411200080598672, var"#sin_pullback#52"{Float64, ProjectTo{Float64, NamedTuple{(), Tuple{}}}}(-0.9899924966004454, ProjectTo{Float64}()))

In [170]:
a_pullback(1)

(NoTangent(), -0.9899924966004454)

In [177]:
a

0.1411200080598672

In [182]:
b, b_pullback = rrule(+, 0.2, a)

(0.3411200080598672, var"#+_pullback#55"{Float64, Float64, ProjectTo{Float64, NamedTuple{(), Tuple{}}}, ProjectTo{Float64, NamedTuple{(), Tuple{}}}}(1.0, 1.0, ProjectTo{Float64}(), ProjectTo{Float64}()))

In [176]:
b

0.3411200080598672

In [178]:
b_pullback(1)

(NoTangent(), 1.0, 1.0)

In [ ]:
# Define rules (alternatively get them for free via `using ChainRules`)
@scalar_rule(sin(x), cos(x))
@scalar_rule(+(x, y), (1.0, 1.0))
@scalar_rule(asin(x), inv(sqrt(1 - x^2)))

#### Find dfoo/dx via rrules
#### First the forward pass, gathering up the pullbacks
x = 3;
a, a_pullback = rrule(sin, x);
b, b_pullback = rrule(+, 0.2, a);
c, c_pullback = rrule(asin, b)

#### Then the backward pass calculating gradients
c̄ = 1;                    # ∂c/∂c
_, b̄ = c_pullback(c̄);     # ∂c/∂b = ∂c/∂b ⋅ ∂c/∂c
_, _, ā = b_pullback(b̄);  # ∂c/∂a = ∂c/∂b ⋅ ∂b/∂a
_, x̄ = a_pullback(ā);     # ∂c/∂x = ∂c/∂a ⋅ ∂a/∂x
x̄                         # ∂c/∂x = ∂foo/∂x
# output
-1.0531613736418153

#### Find dfoo/dx via frules
x = 3;
ẋ = 1;              # ∂x/∂x
nofields = ZeroTangent();  # ∂self/∂self

a, ȧ = frule((nofields, ẋ), sin, x);                    # ∂a/∂x = ∂a/∂x ⋅ ∂x/∂x 
b, ḃ = frule((nofields, ZeroTangent(), ȧ), +, 0.2, a);  # ∂b/∂x = ∂b/∂a ⋅ ∂a/∂x
c, ċ = frule((nofields, ḃ), asin, b);                   # ∂c/∂x = ∂c/∂b ⋅ ∂b/∂x
ċ                                                       # ∂c/∂x = ∂foo/∂x
# output
-1.0531613736418153

#### Find dfoo/dx via FiniteDifferences.jl
using FiniteDifferences
central_fdm(5, 1)(foo, x)
# output
-1.0531613736418257

#### Find dfoo/dx via ForwardDiff.jl
using ForwardDiff
ForwardDiff.derivative(foo, x)
# output
-1.0531613736418153

#### Find dfoo/dx via Zygote.jl
using Zygote
Zygote.gradient(foo, x)
# output
(-1.0531613736418153,)
